In [2]:
%pylab inline
%pip install -U PySuperTuxKart
import torch
import sys, os
import pystk
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('device = ', device)

Populating the interactive namespace from numpy and matplotlib
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 19.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for PySuperTuxKartData: filename=PySuperTuxKartData-1.0.0-py3-none-any.whl size=620700371 sha256=390c34efcdda165428bd8023fe7edc5072c572922961b6adbcbd5f907b8b78f1
  Stored in directory: /root/.cache/pip/wheels/5b/da/db/107e5e579034a950bc34417909d989352cd89aa65988af479f
Successfully built PySuperTuxKartData
device =  cuda


In [3]:
class Rollout:
    def __init__(self, screen_width, screen_height, hd=True, track='lighthouse', render=True):
        # Init supertuxkart
        if not render: # no render
            config = pystk.GraphicsConfig.none()
        elif hd: #HD graphics
            config = pystk.GraphicsConfig.hd()
        else: # Low definition graphics
            config = pystk.GraphicsConfig.ld()
        config.screen_width = screen_width
        config.screen_height = screen_height
        pystk.init(config) # initialize super tux kart

        self.render = render # say if rendering images or not
        race_config = pystk.RaceConfig(track=track) # start race in super tux kart, takes track as input
        self.race = pystk.Race(race_config) # create race
        self.race.start() # start race

    def __call__(self, agent, n_steps=200): # in order to actually race... need agent
        self.race.restart() # restart the level
        self.race.step() # first step (starts rendering engine to produce images)
        data = [] # initialize data
        track_info = pystk.Track() # get track info
        track_info.update() # update track info (static)

        for i in range(n_steps):
            world_info = pystk.WorldState() # world state can change (locaiton of items, other player, etc)
            world_info.update() # update world info

            # Gather world information
            kart_info = world_info.players[0].kart # get info about player's own kart

            agent_data = {'track_info': track_info, 'kart_info': kart_info} # store data in agent info
            if self.render: # if rendering, will also store image into data
                agent_data['image'] = np.array(self.race.render_data[0].image)

            # Act
            action = agent(**agent_data) # determine action to take by inputting the agent data
            agent_data['action'] = action # store action taken in agent data

            # Take a step in the simulation
            self.race.step(action) # take one step in simulator

            # Save all the relevant data
            data.append(agent_data) # save all of the data
        return data

In [4]:
def dummy_agent(**kwargs): # Dummy agent, ignores entire state of super tux kart
    action = pystk.Action() # action
    action.acceleration = 1 # only action is to accelerate
    return action

In [1]:
rollout = Rollout(800, 600) # create rollout object

def rollout_many(many_agents, **kwargs):
    for agent in many_agents:
        yield rollout(agent, **kwargs)

data = rollout(dummy_agent)

NameError: name 'Rollout' is not defined

In [ ]:
def show_video(frames, fps=30):
    import imageio
    from IPython.display import Video, display

    imageio.mimwrite('/tmp/test.mp4', frames, fps=fps, bitrate=10000000)
    display(Video('/tmp/test.mp4', width=800, height=600, embed=True))

show_video([d['image'] for d in data])

In [ ]:
def show_agent(agent, **kwargs): # rolls out agent and displays video
    data = rollout(agent, **kwargs)
    show_video([d['image'] for d in data])

In [ ]:
def less_dummy_agent(**kwargs): # smarter agent that also steers
    action = pystk.Action()
    action.acceleration = 1
    action.steer = -0.6
    return action
show_agent(less_dummy_agent)

In [ ]:
# Let's load a fancy auto-pilot. You'll write one yourself in your homework.
from _auto_pilot import auto_pilot

show_agent(auto_pilot, n_steps=600)